Usage
=====

Basic example
-------------
To use xclim in a project:

In [ ]:
import xclim

To open NetCDF (`.nc`) climate data sets, use [xarray](https://xarray.pydata.org/). For the examples, we will use a tutorial dataset available in xarray.

In [ ]:
import xarray as xr
# For fancy html display, not usually needed 
xr.set_options(display_style='html', display_width=50)

# ds = xr.open_dataset("your_file.nc")
ds = xr.tutorial.open_dataset('air_temperature')
ds

<div class="alert alert-info">

Calculations are performed on the variable of interest present within the data set (in this example, **air**) and not the data set itself, so be sure to specify this when using xclim's indices and other calls.

</div>

To perform a simple climate indice calculation:

In [ ]:
gdd = xclim.indices.growing_degree_days(ds.air, thresh='10.0 degC', freq='MS')
gdd

To plot a time slice of the output, we recommend the following:

In [ ]:
import matplotlib.pyplot
%matplotlib inline

# for a summary statistics histogram
gdd.plot()

In [ ]:
# for a specific geographical coordinate
gdd.isel(lon=-20, lat=10).plot()

In [ ]:
# for a specific time period
gdd.sel(time='2013-07').plot()

For more examples, see the directions suggested by [xarray's plotting documentation](https://xarray.pydata.org/en/stable/plotting.html>)

To save the data as a new NetCDF:

In [ ]:
gdd.to_netcdf('monthly_growing_degree_days_data.nc')

<div class="alert alert-info">
    
You can also save as different formats such as by leveraging the functions already built into the xarray dataset instance. For more information see: [xarray's documentation](https://xarray.pydata.org/en/stable/generated/xarray.Dataset.html)

</div>

Checks and metadata consistency
-------------------------------------------------

The `xclim.atmos`, `xclim.land` and `xclim.seaIce` modules are tools for running error-detection checks and ensuring consistent metadata when running an indice calculation. Some of the processes involve:

* Checking for missing data and incomplete periods.
* Writing [Climate and Forecast Convention](http://cfconventions.org/) compliant metadata based on the variables and indices calculated.
* Identifying periods where missing data significantly impacts the calculation and omits calculations for those periods.
* Appending process history and maintaining the historical provenance of file metadata.

Those modules are best used for producing NetCDF that will be shared with users. See [Climate Indicators](../indicators.rst) for a list of available indicators.

<div class="alert alert-warning">
 
These three modules were written to work with daily data only. They will fail otherwise. To use it with our example dataset, we first need to resample the temperatures to a daily frequency. See [xarray's](http://xarray.pydata.org/en/stable/time-series.html#resampling-and-grouped-operations) and [pandas'](https://pandas.pydata.org/docs/user_guide/timeseries.html#resampling) documentation on the subject.
    
</div>

Recomputing the same climatic indicator as previously on our toy data gives a fuller output. Notice the warnings about missing CF-compliant attributes in the input dataset.

In [ ]:
# Resample to a daily frequency. keep_attrs=True needed to propagate the units attribute
daily_tas = ds.air.resample(time='D').mean(keep_attrs=True)
gdd_atmos = xclim.atmos.growing_degree_days(daily_tas, thresh='10.0 degC', freq='MS')
gdd_atmos